# Business Analysis: What Makes a Successful Film?
## Overview
This project analyzes different characteristics of films to determine what makes films more successful than others. Descriptive analysis of movie data from multiple sources shows that movies from certain studios, certain genres, and certain runtimes perfrom better than others based on profit, revenue and popularity. Movie producers can use this analysis to create movies that will perform successfully.
## Business Problem
A new movie producer wants as much information on the industry before they start making films. The soon to be producer we are providing this analysis for wants to know ways that they can assure movies they make will be successful. We have been tasked with providing actionable insights that will help forge how they make movies.
## Data Sources
The data used for this analysis comes from four different sources and were used for these main DataFrames:
1. [IMDB](https://www.imdb.com/) - `less_90`,`more_90`,`all_runtime`
2. [TheMovieDB](https://www.themoviedb.org/) - `df_gross`
3. [Box Office Mojo](https://www.boxofficemojo.com/) - `df_gross`
4. [The Numbers](https://www.the-numbers.com/) - `budgets`

The most important information used from these sources were movie titles, runtime, revenue, profit, studios, genre, and popularity.

In [1]:
#importing everything we will need
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
from statsmodels.formula.api import ols
from scipy.stats.stats import pearsonr
from sklearn.linear_model import LinearRegression
from statsmodels.stats.diagnostic import linear_rainbow

## Discovering the Relationship Between Total Gross Revenue and Production Studios
### Initial Exploration and Cleaning of the DataFrame (loading `bom.movie_gross.csv.gz`)

In [2]:
df_gross = pd.read_csv("./zippedData/bom.movie_gross.csv.gz")
df_gross['foreign_gross'] = df_gross['foreign_gross'].str.replace(',', '')
df_gross['foreign_gross'] = df_gross['foreign_gross'].astype(float)
df_gross.dropna(subset=['domestic_gross', 'foreign_gross'], inplace=True)
#computing the total gross
df_gross['total_gross']= df_gross['domestic_gross'] + df_gross['foreign_gross']

### Grouping revenue by `studio` and generationg a DataFrame with top 10 production studios.

In [3]:
studio_group = df_gross.groupby('studio')
avg_total = studio_group['total_gross'].mean()
sorted_avg_total = avg_total.sort_values(ascending=False)
top10studios = pd.DataFrame(sorted_avg_total).head(10)

## Discovering the Relationship Between Movie Ratings and Genre

### Transforming the genre codes into the genre names using `genre_ids`.

In [4]:
df_movies = pd.read_csv("./zippedData/tmdb.movies.csv.gz")
#we retreived the genre codes from The Movie Database website and substituted them for 
#the actual genre name.
genre_mapping = {
    28: "Action",
    12: "Adventure",
    16: "Animation",
    35: "Comedy",
    80: "Crime",
    99: "Documentary",
    18: "Drama",
    10751: "Family",
    14: "Fantasy",
    36: "History",
    27: "Horror",
    10402: "Music",
    9648: "Mystery",
    10749: "Romance",
    878: "Science Fiction",
    10770: "TV Movie",
    53: "Thriller",
    10752: "War",
    37: "Western"
}

import ast

def convert_str_list(s):
    try:
        return ast.literal_eval(s)
    except ValueError as e:
        print(f"Error for string: {s}\nError is: {e}")
        return s    # return the original string in case of error
def get_genre_names(genre_codes):
    return [genre_mapping.get(code, "Undefined") for code in genre_codes]

df_movies['genre_ids'] = df_movies['genre_ids'].apply(convert_str_list)
df_movies['genre_names'] = df_movies['genre_ids'].apply(get_genre_names)

df_movies['genre_names'] = df_movies['genre_names'].apply(lambda x: ', '.join(x))

### Breaking down ratings for each individual genre and generating a DataFrame containing the top genres based on weighted rating with respect to `vote_count`.

In [5]:
df_movies['genre_names'] = df_movies['genre_names'].str.split(', ')
df_exploded = df_movies.explode('genre_names')
df_exploded['weighted_vote'] = df_exploded['vote_count'] * df_exploded['vote_average']
df_weighted = df_exploded.groupby('genre_names').agg({'weighted_vote': 'sum', 'vote_count': 'sum'})
df_weighted['average_rating'] = df_weighted['weighted_vote'] / df_weighted['vote_count']

In [6]:
df_weighted = df_weighted.sort_values(by='average_rating', ascending=False)
df_weighted.reset_index(inplace=True)
df_weighted

,genre_names,weighted_vote,vote_count,average_rating
0,History,1431103.3,195600,7.316479
1,War,1024805.5,141336,7.250846
2,Animation,3643815.3,506612,7.192517
3,Documentary,446405.3,62283,7.167370
4,Music,624432.3,87434,7.141756
5,Western,655407.2,92667,7.072714
6,Family,4588380.8,652585,7.031085
7,Drama,14485381.9,2064637,7.015946
8,Adventure,11288628.2,1645828,6.858936
9,Romance,4249937.6,621223,6.841243


## Discovering Relationship between Runtime and Profit
### Initial SQL Data Exploration (Loading in, creating DataFrames)

In [7]:
#Loading in the SQL data 
conn = sqlite3.connect('./zippedData/im.db/im.db')
q = pd.read_sql("""
SELECT *
FROM sqlite_master

""", conn)
q

OperationalError: unable to open database file

Here we can see all the tables contained in this SQL database, `movie_basics` is the table we'll go further into.

In [ ]:
#Exploring the 'movie_basics' table
q = pd.read_sql("""

SELECT *
FROM movie_basics
LIMIT 10

""", conn)
q

In [ ]:
#Creating a DataFrame 'less_90' containing all movies with runtimes less than 90 minutes
less_90 = pd.read_sql("""

SELECT DISTINCT primary_title, runtime_minutes
FROM movie_basics
WHERE runtime_minutes < 90

""", conn)
less_90

In [ ]:
##Creating a DataFrame 'more_90' containing all movies with runtimes more than 90 minutes
more_90 = pd.read_sql("""

SELECT DISTINCT primary_title, runtime_minutes
FROM movie_basics
WHERE runtime_minutes > 90

""", conn)
more_90

### Cleaning the `more_90` and `less_90` DataFrames

In [ ]:
# Getting rid of all duplicates, no null values to worry about
more_90.drop_duplicates(subset='primary_title', keep=False, inplace=True)
less_90.drop_duplicates(subset='primary_title', keep=False, inplace=True)

### Loading in `budgets` and Merging with `more_90` and `less_90`

In [ ]:
# Loading in a new DataFrame
budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

In [ ]:
budgets.head()

In [ ]:
# Getting rid of all duplicates, no null values to worry about
budgets.drop_duplicates(subset='movie', keep=False, inplace=True)

In [ ]:
# Creating 2 new DataFrames by merging more_90 with budgets and less_90 with budgets.
# Merged using the titles of shared films between the DataFrames.
more_90_budgets = more_90.merge(budgets, how='inner', left_on='primary_title', right_on='movie')
less_90_budgets = less_90.merge(budgets, how='inner', left_on='primary_title', right_on='movie')

In [ ]:
more_90_budgets.head()

In [ ]:
less_90_budgets.head()

### Cleaning `more_90_budgets` and `less_90_budgets`

In [ ]:
# For columns that will be used, getting rid of dollar signs and commas.
# After this they can be turned into floats.
more_90_budgets['production_budget'] = more_90_budgets['production_budget'].str.strip('$')
more_90_budgets['production_budget'] = more_90_budgets['production_budget'].str.replace(',','')
more_90_budgets['worldwide_gross'] = more_90_budgets['worldwide_gross'].str.strip('$')
more_90_budgets['worldwide_gross'] = more_90_budgets['worldwide_gross'].str.replace(',' , '')
more_90_budgets['worldwide_gross'] = more_90_budgets['worldwide_gross'].astype(float)
more_90_budgets['production_budget'] = more_90_budgets['production_budget'].astype(float)

In [ ]:
# Creating a new column named 'profit' using 'worldwide_gross' minus 'production_budget'
more_90_budgets['profit'] = more_90_budgets['worldwide_gross'] - more_90_budgets['production_budget']

#### Repeat the same cleaning process for `less_90_budgets`

In [ ]:
# For columns that will be used, getting rid of dollar signs and commas.
# After this they can be turned into floats.
less_90_budgets['production_budget'] = less_90_budgets['production_budget'].str.strip('$')
less_90_budgets['production_budget'] = less_90_budgets['production_budget'].str.replace(',','')
less_90_budgets['worldwide_gross'] = less_90_budgets['worldwide_gross'].str.strip('$')
less_90_budgets['worldwide_gross'] = less_90_budgets['worldwide_gross'].str.replace(',' , '')
less_90_budgets['worldwide_gross'] = less_90_budgets['worldwide_gross'].astype(float)
less_90_budgets['production_budget'] = less_90_budgets['production_budget'].astype(float)

In [ ]:
# Creating a new column named 'profit' using 'worldwide_gross' minus 'production_budget'
less_90_budgets['profit'] = less_90_budgets['worldwide_gross'] - less_90_budgets['production_budget']

## Viewing Relationship of `profit` vs. `runtime_minutes` for Films < 90 mins and Films > 90 mins

In [ ]:
# Scatter plot for movies shorter than 90 minutes
less_90_budgets.plot.scatter(x='runtime_minutes', y='profit');

In [ ]:
# Scatter plot for movies longer than 90 minutes
more_90_budgets.plot.scatter(x='runtime_minutes', y='profit');

### The means below show a large difference between the average profits. It seems like films with runtimes longer than 90 minutes make more money on average than movies less than 90 minutes.

In [ ]:
# Finding the mean profit for 'less_90_budgets' and 'more_90_budgets' 
less_90_budgets['profit'].mean(), more_90_budgets['profit'].mean()

### We can find out if it's statistically significant with a 2 sample T-test. First, let's state our null hypothesis, alternative hypothesis, and alpha level.
#### H0: Films with a runtime less than 90 mins and films with a runtime greater than 90 minutes return the same amount of profit on average.
#### H1: Films with a runtime less than 90 mins and films with a runtime greater than 90 minutes return a different amount of profit on average.
#### Alpha = 0.05 
#### Now, let's run the test and check the results.

In [ ]:
# Running a 2 sample ttest
# alpha = 0.05
tstat, pvalue = stats.ttest_ind(less_90_budgets['profit'], more_90_budgets['profit'], equal_var=False)
tstat, pvalue

### The pvalue is < 0.05 meaning we reject the null hypothesis. This concludes that on average, films with a runtime less than 90 mins and films with a runtime greater than 90 minutes return a different amount of profit on average.
#### More specifically, films that are longer than 90 minutes return a greater profit on average.

## Exploring how Runtime influences Popularity
### Creating the `all_runtime`

In [ ]:
# Creating a new DataFrame that contains all runtime lengths
all_runtime = pd.read_sql("""

SELECT DISTINCT primary_title, runtime_minutes
FROM movie_basics


""", conn)
all_runtime

### Cleaning `all_runtime`

In [ ]:
# Getting rid of duplicates
all_runtime.drop_duplicates(subset='primary_title', keep=False, inplace=True)

### Merging `all_runtime` with `budgets`

In [ ]:
# Merging using the titles of shared films between the DataFrames.
all_runtime_budgets = all_runtime.merge(budgets, how='inner', left_on='primary_title', right_on='movie')

### Cleaning `all_runtime_budgets` 
Performing the same cleaning processes that were done for `more_90_budgets` and `less_90_budgets`

In [ ]:
# For columns that will be used, getting rid of dollar signs and commas.
# After this they can be turned into floats.
all_runtime_budgets['production_budget'] = all_runtime_budgets['production_budget'].str.strip('$')
all_runtime_budgets['production_budget'] = all_runtime_budgets['production_budget'].str.replace(',','')
all_runtime_budgets['production_budget'] = all_runtime_budgets['production_budget'].astype(float)
all_runtime_budgets['worldwide_gross'] = all_runtime_budgets['worldwide_gross'].str.strip('$')
all_runtime_budgets['worldwide_gross'] = all_runtime_budgets['worldwide_gross'].str.replace(',','')
all_runtime_budgets['worldwide_gross'] = all_runtime_budgets['worldwide_gross'].astype(float)

In [ ]:
# Creating a new column named 'profit' using 'worldwide_gross' minus 'production_budget'
all_runtime_budgets['profit'] = all_runtime_budgets['worldwide_gross'] - all_runtime_budgets['production_budget']

### Creating `df_movies2`

In [ ]:
# Taking only the movie title and popularity rating from 'df_movies' DataFrame
df_movies2 = df_movies[['original_title' , 'popularity']]

### Merging `df_movies2` and `budgets` into `df_movies2_budgets`


In [ ]:
# Merged using the titles of the movies
df_movies2_budgets = df_movies2.merge(all_runtime_budgets, how='inner', left_on='original_title', right_on='movie')

### Cleaning `df_movies2_budgets`

In [ ]:
# Getting rid of null values
df_movies2_budgets.dropna(subset=['runtime_minutes'], inplace=True)

### Visually exploring the relationship between `runtime_minutes` and `popularity`

In [ ]:
# Creating scatter plot
df_movies2_budgets.plot.scatter(x='runtime_minutes', y='popularity');

Above it's evident that there are some outliers that will poorly affect a linear regression. There looks to be about 8 values at the lowest end of `runtime_minutes` that are less than ~70 minutes. Then there are the two highest points with `popularity` greater than 70. It is in our best interest to get rid of these outliers to improve the results of a linear regression.

### Getting rid of the outliers

In [ ]:
# Only keeping movies that have a runtime greater than or equal to 70 minutes 
df_movies2_budgets = df_movies2_budgets[df_movies2_budgets['runtime_minutes'] >= 70]

In [ ]:
# Only keeping movies that have a popularity score of less than 65
df_movies2_budgets = df_movies2_budgets[df_movies2_budgets['popularity'] < 65]

### Visually exploring the relationship now that the outliers are gone

In [ ]:
# Creating scatter plot
df_movies2_budgets.plot.scatter(x='runtime_minutes', y='popularity');

The scatter plot above looks a lot better.

### Rainbow test of linearity
To make sure our data is linear (one of the four assumptions made when running a linear regression) we can run a rainbow test. The null hypothesis of this test is that the data is linear. We will set our alpha level at 0.01.

In [ ]:
# Running the rainbow test
model = ols("popularity ~ runtime_minutes", data=df_movies2_budgets).fit()
F_statistic, p_value = linear_rainbow(res=model, frac=0.5)
F_statistic, p_value

The pvalue is > 0.01, meaning we fail to reject the null hypothesis and we can safely assume that our data is linear based on our test results.

### Linear Regression for `runtime_minutes` and `popularity`

In [ ]:
# Running the linear regression and printing the R-Squared value
X = df_movies2_budgets[['runtime_minutes']]  
y = df_movies2_budgets['popularity']  
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
r_squared = model.score(X, y)
print("R-squared value: ", r_squared)

This R-Squared value means that about 19% of the popularity score a movie receives is explained by the runtime of the movie.

## Conclusion
Based on our data analysis we have three final recommendations.
1. Work with one of our top studios because they have the highest gross revenues.
2. Make movies under one of our top genres because these types of movies earn the highest ratings.
3. Make movies that are longer than 90 minutes and aim around 120 minutes in length. These movies return the highest profits.
## Next Steps
- Explore which age rating (P, PG, PG-13, R) would be best
- Explore which actors bring in the most revenue
- Explore which directors are best for -insert genre rec here-